# Cálculo de derivadas evaporación y graficación

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

**Librerías**

In [1]:
import pandas as pd
import numpy as np
import datetime
import statistics
import glob
import os
import csv
import re
import gc
import calendar
from collections import deque
from datetime import timedelta
from scipy import stats
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference
from openpyxl.chart import ScatterChart, Reference, Series
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

____

### Pruebas unitarias

#### Datos con QC

In [24]:
# Función para procesar frecuencias
def process_frequencies(df, columna_fecha, freq_csv_path, porc_min=0.7):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Cargar el archivo de frecuencias
    freqinst200b = pd.read_csv(freq_csv_path, encoding='latin-1', sep=';')

    # Definir el diccionario de frecuencias y cantidades esperadas
    frecuencias = {
        'min': {'cant_esperd_h': 60, 'cant_esperd_d': 1440, 'cant_esperd_m': 43200, 'cant_esperd_a': 518400, 'minutos': 1},
        '2min': {'cant_esperd_h': 30, 'cant_esperd_d': 720, 'cant_esperd_m': 21600, 'cant_esperd_a': 259200, 'minutos': 2},        
        '5min': {'cant_esperd_h': 12, 'cant_esperd_d': 288, 'cant_esperd_m': 8640, 'cant_esperd_a': 103680, 'minutos': 5},
        '10min': {'cant_esperd_h': 6, 'cant_esperd_d': 144, 'cant_esperd_m': 4320, 'cant_esperd_a': 51840, 'minutos': 10},
        'h': {'cant_esperd_h': 1, 'cant_esperd_d': 24, 'cant_esperd_m': 720, 'cant_esperd_a': 8640}
    }

    # Obtener el valor de la estación
    station_value = df['Station'].values[0]
    freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == station_value]
    periodos = freqinst200b_station['FreqInf'].values[0]
    print(periodos)

    if pd.isna(periodos):
        try:
            periodos = pd.infer_freq(df[columna_fecha][-25:])
            print(periodos)
            if periodos is None:
                project_value = freqinst100b_station['Instituc'].values[0]
                periodos = {'CENICAFE': '5min', 'IDEAM': '10min', 'CAR': 'h', 'IDIGER': 'min'}.get(project_value, 'min')
                print(f"Frecuencia inferida para {df['Station']} es None. Se determina según entidad {project_value}")
                
                #return None, None, None, None, None, None
        except ValueError as e:
            print(f'Error al inferir la frecuencia en el archivo {df}: {str(e)}')
            return None, None, None, None, None, None

    # Obtener las cantidades esperadas y el offset en minutos
    cant_esperd_h = frecuencias[periodos]['cant_esperd_h']
    cant_esperd_d = frecuencias[periodos]['cant_esperd_d']
    cant_esperd_m = frecuencias[periodos]['cant_esperd_m']
    cant_esperd_a = frecuencias[periodos]['cant_esperd_a']
    if periodos == 'h':
        pass
    else:
        minutoffset = frecuencias[periodos]['minutos']

    # Ajustar la hora de cada registro
    df_c = df.copy()
    if periodos == 'h':
        df_c[columna_fecha] = df_c[columna_fecha] #- pd.Timedelta(hours=1)
    else:
        df_c[columna_fecha] = df_c[columna_fecha] - pd.Timedelta(minutes=minutoffset)

    # Establecer la columna de fecha como índice
    df_c.set_index(columna_fecha, inplace=True)

    # Función para verificar si un día, mes o año tiene suficientes datos
    def complet_hora(sub_df):
        return len(sub_df) >= cant_esperd_h * porc_min
    
    def complet_dia(sub_df):
        return len(sub_df) >= cant_esperd_d * porc_min

    def complet_mes(sub_df):
        return len(sub_df) >= cant_esperd_m * porc_min

    def complet_anio(sub_df):
        return len(sub_df) >= cant_esperd_a * porc_min

    return (df_c, periodos, cant_esperd_h, cant_esperd_d, cant_esperd_m, cant_esperd_a,
           complet_hora, complet_dia, complet_mes, complet_anio)
    #complet_hora, complet_dia, complet_mes, complet_anio

# Uso funciones de frecuencias
df_example = pd.read_csv('../../OE_3_QC_Variables/5_Evaporacion/RawUnmodified_EV/Estacion_0021205970.csv', encoding='latin-1')#, dtype={'Estado_Anterior':str})
if 'Estado' in df_example.columns:
    df_example = df_example[~df_example['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO','0PAT','0PER']]))]
    
columna_fecha = 'Fecha'
freq_csv_path = '../../OE_3_QC_Variables/5_Evaporacion/EMAEV_LatLonEntFreq.csv'

In [30]:
## V1, primero, cálculo de evaporación instantánea, luego
## Derivadas diarias, mensuales y anuales
# Evaporación
df_example_c = df_example.copy()

def EV_I(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
    """
    Calcula la evaporación en un tanque a partir de datos de nivel, considerando solo las disminuciones de nivel.
    """
    # Asegurarnos de que la columna de fecha sea datetime
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Ordenar por fecha por seguridad
    df = df.sort_values(by=columna_fecha).reset_index(drop=True)
   
    # Calcular la diferencia entre niveles consecutivos
    df['Diferencia'] = df[columna_valor].diff()
    
    # Filtrar solo las disminuciones (diferencias negativas)
    df['Evaporacion'] = df['Diferencia'].where(df['Diferencia'] < 0, 0).abs()
    
    return df[['Station','Fecha','Evaporacion']] # evaporacion_total,

# Se llama el archivo original según si es dato crudo o con qc
ev_i = EV_I(df_example_c)
ev_i_c = ev_i.copy()

def EV_TT_H(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Se llama la función de procesamiento de frecuencias
    df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
    
    if periodos == 'H':
        return df[['Station','Fecha','Evaporacion']]

    if df_c is None or complet_hora is None:
        return None

    # Se filtran los datos que tienen la complititud mínima
    df_filtrado = df_c.groupby([df_c.index.hour]).filter(complet_hora)
    ev_tt_h = df_filtrado[[columna_valor]].resample('h').sum().round(4)
    ev_tt_h[columna_valor] = ev_tt_h[columna_valor].where(
        df_filtrado[columna_valor].resample('h').count() >= cant_esperd_h * porc_min,
        other=float('nan'))

    return ev_tt_h

# Ejemplo de uso de la función
ev_tt_h = EV_TT_H(ev_i)


ev_tt_h_c = ev_tt_h.copy()
def EV_TT_D(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Revisar resultado anterior
    if df.empty:
        return df
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Establecer la columna de fecha como índice
    df.set_index(columna_fecha, inplace=True)
    cant_esperd_d = 24
    def complet_dia(sub_df):
        return len(sub_df) >= cant_esperd_d * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
    # Calcular la suma diaria de evaporación
    ev_tt_d = df_filtrado[[columna_valor]].resample('D').sum().round(4)
    # Aplicar filtro de completitud diaria
    ev_tt_d[columna_valor] = ev_tt_d[columna_valor].where(
        df_filtrado[columna_valor].resample('D').count() >= cant_esperd_d * porc_min,
        other=float('nan'))
    
    return ev_tt_d

ev_tt_d = EV_TT_D(ev_tt_h_c)

ev_tt_d_c = ev_tt_d.copy()
def EV_TT_M(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Revisar resultado anterior
    if df.empty:
        return df
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Establecer la columna de fecha como índice
    df.set_index(columna_fecha, inplace=True)

    # Cantidad días mes
    days_in_month = df.index.to_series().dt.days_in_month
    days_in_month = days_in_month.resample('ME').first()
    
    # Función para verificar si una hora específica tiene suficientes datos
    def complet_mes(sub_df):
        mes = sub_df.index[0].month
        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
        return len(sub_df) >= total_esperado * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
    # Calcular la suma diaria de evaporación
    ev_tt_m = df_filtrado[[columna_valor]].resample('ME').sum().round(4)

    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
    counts = df.groupby([df.index.year, df.index.month])[columna_valor].count()
    counts.index = pd.to_datetime(['{}-{}'.format(i[0], i[1]) for i in counts.index], format='%Y-%m')
    counts = counts.resample('ME').sum()
    ev_tt_m[columna_valor] = ev_tt_m[columna_valor].where(counts >= days_in_month * porc_min, other=float('nan'))
    
    return ev_tt_m

ev_tt_m = EV_TT_M(ev_tt_d_c)


ev_tt_m_c = ev_tt_m.copy()
def EV_TT_A(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Revisar resultado anterior
    if df.empty:
        return df
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
   
    # Establecer la columna de fecha como índice
    df.set_index(columna_fecha, inplace=True)
    
    cant_esperd_a = 12
    def complet_anio(sub_df):
        return len(sub_df) >= cant_esperd_a * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
    # Calcular la suma diaria de evaporación
    ev_tt_a = df_filtrado[[columna_valor]].resample('YE').sum().round(4)

    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
    counts = df.groupby(df.index.year)[columna_valor].count()
    counts.index = pd.to_datetime(['{}-01'.format(i) for i in counts.index], format='%Y-%m')
    counts = counts.resample('YE').sum()
    ev_tt_a[columna_valor] = ev_tt_a[columna_valor].where(counts >= 12 * porc_min, other=float('nan'))

    return ev_tt_a

ev_tt_a = EV_TT_A(ev_tt_m_c)

## Derivadas medias horarias diarias, mensuales y anuales
# Evaporación media horaria
def EV_MEDIA_H(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Se llama la función de procesamiento de frecuencias
    df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
    
    if periodos == 'H':
        return df_c[['Station','Fecha','Evaporacion']]

    if df_c is None or complet_hora is None:
        return None

    # Se filtran los datos que tienen la complititud mínima
    df_filtrado = df_c.groupby([df_c.index.hour]).filter(complet_hora) #dfC.groupby([dfC.index.date]).filter(complet_dia)
    ev_media_h = df_filtrado[[columna_valor]].resample('h').mean().round(4)

    return ev_media_h

ev_media_h = EV_MEDIA_H(ev_i_c)

# Evaporación media diaria
ev_media_h_c = ev_media_h.copy()
def EV_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Revisar resultado anterior
    if df.empty:
        return df
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Establecer la columna de fecha como índice
    df.set_index(columna_fecha, inplace=True)
    cant_esperd_d = 24
    def complet_dia(sub_df):
        return len(sub_df) >= cant_esperd_d * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
    # Calcular la suma diaria de evaporación
    ev_media_d = df_filtrado[[columna_valor]].resample('D').mean().round(4)
    # Aplicar filtro de completitud diaria
    ev_media_d[columna_valor] = ev_media_d[columna_valor].where(
        df_filtrado[columna_valor].resample('D').count() >= cant_esperd_d * porc_min,
        other=float('nan'))

    return ev_media_d

ev_media_d = EV_MEDIA_D(ev_media_h_c)

# Evaporación media mensual
ev_media_d_c = ev_media_d.copy()
def EV_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Revisar resultado anterior
    if df.empty:
        return df
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Establecer la columna de fecha como índice
    df.set_index(columna_fecha, inplace=True)

    # Cantidad días mes
    days_in_month = df.index.to_series().dt.days_in_month
    days_in_month = days_in_month.resample('ME').first()
    
    # Función para verificar si una hora específica tiene suficientes datos
    def complet_mes(sub_df):
        mes = sub_df.index[0].month
        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
        return len(sub_df) >= total_esperado * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
    # Calcular la suma diaria de evaporación
    ev_media_m = df_filtrado[[columna_valor]].resample('ME').mean().round(4)

    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
    counts = df.groupby([df.index.year, df.index.month])[columna_valor].count()
    counts.index = pd.to_datetime(['{}-{}'.format(i[0], i[1]) for i in counts.index], format='%Y-%m')
    counts = counts.resample('ME').sum()
    ev_media_m[columna_valor] = ev_media_m[columna_valor].where(counts >= days_in_month * porc_min, other=float('nan'))
    
    return ev_media_m

ev_media_m = EV_MEDIA_M(ev_media_d_c)

# Evaporación media anual
ev_media_m_c = ev_media_m.copy()
def EV_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
    df.reset_index(inplace=True)
    # Revisar resultado anterior
    if df.empty:
        return df
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
   
    # Establecer la columna de fecha como índice
    df.set_index(columna_fecha, inplace=True)
    
    cant_esperd_a = 12
    def complet_anio(sub_df):
        return len(sub_df) >= cant_esperd_a * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
    # Calcular la suma diaria de evaporación
    ev_media_a = df_filtrado[[columna_valor]].resample('YE').mean().round(4)

    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
    counts = df.groupby(df.index.year)[columna_valor].count()
    counts.index = pd.to_datetime(['{}-01'.format(i) for i in counts.index], format='%Y-%m')
    counts = counts.resample('YE').sum()
    ev_media_a[columna_valor] = ev_media_a[columna_valor].where(counts >= 12 * porc_min, other=float('nan'))

    return ev_media_a

ev_media_a = EV_MEDIA_A(ev_media_m_c)

# ###-- Derivados máximos y mínimos
# #Humedad relativa del aire a 10 cm  mínima diaria
# df_example_c = df_example.copy()
# def EV_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
#     # Convertir la columna de fecha a datetime si aún no lo es
#     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
#         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
#     # Se llama la función de procesamiento de frecuencias
#     df_c, complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

#     if df_c is None or complet_dia is None:
#         return None

#     # Filtrar los días que tienen suficientes datos
#     df_c['Fecha_temp'] = df_c.index
#     dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
    
#     # Filtrar el DataFrame original para incluir solo los días válidos
#     dias_validos = pd.to_datetime(dias_validos).normalize()
#     df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
    
#     # Encontrar el valor mínimo por cada día válido
#     idx_minimos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmin()
#     mn_d = df_filtrado.loc[idx_minimos_dia]
    
#     # Eliminar la columna temporal antes de retornar el resultado
#     if 'Fecha_temp' in mn_d.columns:
#         mn_d.drop(columns=['Fecha_temp'], inplace=True)
    
#     return mn_d[[columna_valor]]

# df_mn_d = HRS30_MN_D(df_example_c)

# # Humedad relativa del aire a 10 cm  máxima diaria
# df_example_c = df_example.copy()
# def HRS30_MX_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
#     # Convertir la columna de fecha a datetime si aún no lo es
#     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
#         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
#     # Se llama la función de procesamiento de frecuencias
#     df_c, complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

#     if df_c is None or complet_dia is None:
#         return None

#     # Filtrar los días que tienen suficientes datos
#     df_c['Fecha_temp'] = df_c.index
#     dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
#     # Filtrar el DataFrame original para incluir solo los días válidos
#     dias_validos = pd.to_datetime(dias_validos).normalize()
#     df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
#     # Encontrar el valor mínimo por cada día válido
#     idx_maximos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmax()
#     mx_d = df_filtrado.loc[idx_maximos_dia]
    
#     # Eliminar la columna temporal antes de retornar el resultado
#     if 'Fecha_temp' in mx_d.columns:
#         mx_d.drop(columns=['Fecha_temp'], inplace=True)
    
#     return mx_d[[columna_valor]]

# df_mx_d = HRS30_MX_D(df_example_c)

# # Humedad relativa del aire a 10 cm  mínima mensual
# df_example_c = df_example.copy()
# def HRS30_MN_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
#     # Convertir la columna de fecha a datetime si aún no lo es
#     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
#         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
#     # Se llama la función de procesamiento de frecuencias
#     df_c, _, complet_mes, _ = process_frequencies(df, columna_fecha, freq_csv_path,porc_min)

#     if df_c is None or complet_mes is None:
#         return None

#     # Filtrar los meses que tienen suficientes datos
#     df_c['Fecha_temp'] = df_c.index
#     # Eliminar duplicados en el índice
#     df_c = df_c[~df_c.index.duplicated(keep='first')]
#     meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')

#     # Filtrar el DataFrame original para incluir solo los meses válidos
#     df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]

#     # Encontrar el valor mínimo por cada mes válido
#     idx_minimos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmin()
#     min_m = df_filtrado.loc[idx_minimos_mes]

#     # Eliminar las columnas temporales antes de retornar el resultado
#     if 'Fecha_temp' in min_m.columns:
#         min_m.drop(columns=['Fecha_temp'], inplace=True)

#     return min_m[[columna_valor]]
    
# df_mn_m = HRS30_MN_M(df_example_c)

# # Humedad relativa del aire a 10 cm  máxima mensual
# df_example_c = df_example.copy()
# def HRS30_MX_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
#     # Convertir la columna de fecha a datetime si aún no lo es
#     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
#         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
#     # Se llama la función de procesamiento de frecuencias
#     df_c, _, complet_mes, _ = process_frequencies(df, columna_fecha, freq_csv_path,porc_min)

#     if df_c is None or complet_mes is None:
#         return None

#     # Filtrar los meses que tienen suficientes datos
#     df_c['Fecha_temp'] = df_c.index
#     # Eliminar duplicados en el índice
#     df_c = df_c[~df_c.index.duplicated(keep='first')]
#     meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')

#     # Filtrar el DataFrame original para incluir solo los meses válidos
#     df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]

#     # Encontrar el valor mínimo por cada mes válido
#     idx_maximos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmax()
#     max_m = df_filtrado.loc[idx_maximos_mes]

#     # Eliminar las columnas temporales antes de retornar el resultado
#     if 'Fecha_temp' in max_m.columns:
#         max_m.drop(columns=['Fecha_temp'], inplace=True)

#     return max_m[[columna_valor]]
    
# df_mx_m = HRS30_MX_M(df_example_c)

# # Humedad relativa del aire a 10 cm mínima anual
# df_example_c = df_example.copy()
# def HRS30_MN_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
#     # Convertir la columna de fecha a datetime si aún no lo es
#     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
#         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
#     # Se llama la función de procesamiento de frecuencias
#     df_c, _, _, complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

#     if df_c is None or complet_anio is None:
#         return None

#     # Filtrar los meses que tienen suficientes datos
#     df_c['Fecha_temp'] = df_c.index
#     # Eliminar duplicados en el índice
#     df_c = df_c[~df_c.index.duplicated(keep='first')]
#     anios_validos = df_c.groupby(df_c.index.to_period('A')).filter(complet_anio).index.to_period('A')

#     # Filtrar el DataFrame original para incluir solo los meses válidos
#     df_filtrado = df_c[df_c.index.to_period('A').isin(anios_validos)]

#     # Encontrar el valor mínimo por cada mes válido
#     idx_minimos_anio = df_filtrado.groupby(df_filtrado.index.to_period('A'))[columna_valor].idxmin()
#     min_a = df_filtrado.loc[idx_minimos_anio]

#     # Eliminar las columnas temporales antes de retornar el resultado
#     if 'Fecha_temp' in min_a.columns:
#         min_a.drop(columns=['Fecha_temp'], inplace=True)

#     return min_a[[columna_valor]]
    
# df_mn_a = HRS30_MN_A(df_example_c)

# # Humedad relativa del aire a 10 cm  máxima anual
# df_example_c = df_example.copy()
# def HRS30_MX_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
#     # Convertir la columna de fecha a datetime si aún no lo es
#     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
#         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
#     # Se llama la función de procesamiento de frecuencias
#     df_c, _, _, complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

#     if df_c is None or complet_anio is None:
#         return None

#     # Filtrar los meses que tienen suficientes datos
#     df_c['Fecha_temp'] = df_c.index
#     # Eliminar duplicados en el índice
#     df_c = df_c[~df_c.index.duplicated(keep='first')]
#     anios_validos = df_c.groupby(df_c.index.to_period('A')).filter(complet_anio).index.to_period('A')

#     # Filtrar el DataFrame original para incluir solo los meses válidos
#     df_filtrado = df_c[df_c.index.to_period('A').isin(anios_validos)]

#     # Encontrar el valor mínimo por cada mes válido
#     idx_minimos_anio = df_filtrado.groupby(df_filtrado.index.to_period('A'))[columna_valor].idxmax()
#     max_a = df_filtrado.loc[idx_minimos_anio]

#     # Eliminar las columnas temporales antes de retornar el resultado
#     if 'Fecha_temp' in max_a.columns:
#         max_a.drop(columns=['Fecha_temp'], inplace=True)

#     return max_a[[columna_valor]]
    
# df_mx_a = HRS30_MN_A(df_example_c)

h
h


In [ ]:
## -- Gráficas y export en excel
# Se crea un nuevo archivo Excel con openpyxl
wb = Workbook()
sheets = {
    'EV_I': ev_i, 'EV_TT_H': ev_tt_h,
    'EV_TT_D': ev_tt_d, 'EV_TT_M': ev_tt_m,
    'EV_TT_A': ev_tt_a, 'EV_MEDIA_H': ev_media_h,
    'EV_MEDIA_D': ev_media_d, 'EV_MEDIA_M': ev_media_m,
    'EV_MEDIA_A': ev_media_a,
    #'HRS30_MN_M': df_mn_m,
    #'HRS30_MX_M': df_mx_m, 'HRS30_MN_A': df_mn_a,
    #'HRS30_MX_A': df_mx_a 
}

# Si el workbook todavía tiene la hoja por defecto, se elimina
if "Sheet" in wb.sheetnames:
    del wb["Sheet"]

for sheet_name, data in sheets.items():
    ws = wb.create_sheet(title=sheet_name)
    
    # Agregamos los datos al Excel
    for r_idx, row in enumerate(dataframe_to_rows(data, index=True, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)
    
    # Crear una gráfica
    chart = LineChart()
    chart.title = sheet_name
    chart.style = 5
    chart.y_axis.title = 'Evaporación (mm)'
    chart.x_axis.title = 'Fecha'
    
    # Establecer datos para la gráfica
    max_row = ws.max_row
    values = Reference(ws, min_col=2, min_row=2, max_col=2, max_row=max_row)
    dates = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=max_row)
    chart.add_data(values, titles_from_data=True)
    chart.set_categories(dates)
    
    # Quitar la leyenda
    chart.legend = None

    # Cambiar el grosor de la línea a 0.5 puntos (equivalente a 50 centésimas de punto)
    for series in chart.series:
        series.graphicalProperties.line.width = 50
        series.graphicalProperties.line.solidFill = "3498db"  # Marrón
        
    # Posicionar la gráfica en el Excel
    ws.add_chart(chart, "E3")

# Guardar el archivo Excel
wb.save("Agreg_graf_EV_AUT.xlsx")

## Función para cálculo masivo de derivadas y generación de gráficas

In [3]:
# Función para procesar frecuencias
def process_frequencies(df, columna_fecha, freq_csv_path, porc_min=0.7):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Cargar el archivo de frecuencias
    freqinst200b = pd.read_csv(freq_csv_path, encoding='latin-1', sep=';')

    # Definir el diccionario de frecuencias y cantidades esperadas
    frecuencias = {
        'min': {'cant_esperd_h': 60, 'cant_esperd_d': 1440, 'cant_esperd_m': 43200, 'cant_esperd_a': 518400, 'minutos': 1},
        '2min': {'cant_esperd_h': 30, 'cant_esperd_d': 720, 'cant_esperd_m': 21600, 'cant_esperd_a': 259200, 'minutos': 2},
        '5min': {'cant_esperd_h': 12, 'cant_esperd_d': 288, 'cant_esperd_m': 8640, 'cant_esperd_a': 103680, 'minutos': 5},
        '10min': {'cant_esperd_h': 6, 'cant_esperd_d': 144, 'cant_esperd_m': 4320, 'cant_esperd_a': 51840, 'minutos': 10},
        'h': {'cant_esperd_h': 1, 'cant_esperd_d': 24, 'cant_esperd_m': 720, 'cant_esperd_a': 8640}
    }

    # Obtener el valor de la estación
    station_value = df['station'].values[0] # Station
    freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == station_value]
    periodos = freqinst200b_station['FreqInf'].values[0]

    if pd.isna(periodos):
        try:
            periodos = pd.infer_freq(df[columna_fecha][-25:])
            print(periodos)
            if periodos is None:
                project_value = freqinst100b_station['Instituc'].values[0]
                periodos = {'CENICAFE': '5min', 'IDEAM': '10min', 'CAR': 'h', 'IDIGER': 'min'}.get(project_value, 'min')
                print(f"Frecuencia inferida para {df['station']} es None. Se determina según entidad {project_value}") #Station
                
                #return None, None, None, None, None, None
        except ValueError as e:
            print(f'Error al inferir la frecuencia en el archivo {df}: {str(e)}')
            return None, None, None, None, None, None

    # Obtener las cantidades esperadas y el offset en minutos
    cant_esperd_h = frecuencias[periodos]['cant_esperd_h']
    cant_esperd_d = frecuencias[periodos]['cant_esperd_d']
    cant_esperd_m = frecuencias[periodos]['cant_esperd_m']
    cant_esperd_a = frecuencias[periodos]['cant_esperd_a']
    if periodos == 'h':
        pass
    else:
        minutoffset = frecuencias[periodos]['minutos']

    # Ajustar la hora de cada registro
    df_c = df.copy()
    if periodos == 'h':
        df_c[columna_fecha] = df_c[columna_fecha] #- pd.Timedelta(hours=1)
    else:
        df_c[columna_fecha] = df_c[columna_fecha] - pd.Timedelta(minutes=minutoffset)

    # Establecer la columna de fecha como índice
    df_c.set_index(columna_fecha, inplace=True)

    # Función para verificar si un día, mes o año tiene suficientes datos
    def complet_hora(sub_df):
        return len(sub_df) >= cant_esperd_h * porc_min
    
    def complet_dia(sub_df):
        return len(sub_df) >= cant_esperd_d * porc_min

    def complet_mes(sub_df):
        return len(sub_df) >= cant_esperd_m * porc_min

    def complet_anio(sub_df):
        return len(sub_df) >= cant_esperd_a * porc_min

    return df_c, periodos, cant_esperd_h, cant_esperd_d, cant_esperd_m, cant_esperd_a, complet_hora, complet_dia, complet_mes, complet_anio

# # Uso funciones de frecuencias
# df_example = pd.read_csv('../../OE_3_QC_Variables/2_HumedadRelativa/Test_QC/Estacion_0011115501.csv', encoding='latin-1')#, dtype={'Estado_Anterior':str})
# if 'Estado' in df_example.columns:
#     df_example = df_example[df_example['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))]    
columna_fecha = 'event_time' #Fecha
freq_csv_path = '../../OE_3_QC_Variables/5_Evaporacion/EMAEV_LatLonEntFreq.csv'

In [7]:
# Función para cálculo de derivadas de varios archivos en una sola carpeta
def calc_deriv_EV(carpeta, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
            
            for chunk in reader:
                # Se generan dataframes analizados
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                try:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                except ValueError:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')

                if 'Estado' in chunk.columns:
                    try:
                        dfC = chunk[~chunk['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO','0PAT','0PER']]))]
                        dfC_c = dfC.copy()
                        station_value = dfC_c['Station'].values[0]
                    except IndexError:
                        print(f"Error en el archivo {archivo}: dfC está vacío. Saltando al siguiente archivo.")
                        continue  # Sale del bucle de chunks y continúa con el siguiente archivo
                else:
                    chunk_c = chunk.copy()
                    station_value = chunk_c['Station'].values[0]

                # Cálculo de la evaporación instantánea
                def EV_I(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                    """
                    Calcula la evaporación en un tanque a partir de datos de nivel, considerando solo las disminuciones de nivel.
                    """
                    # Asegurarnos de que la columna de fecha sea datetime
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

                    # Asegurarse de que 'periodos' tenga un número antes de la unidad
                    if periodos.isalpha():
                        periodos = '1' + periodos
                        
                    # Ordenar por fecha por seguridad
                    df = df.sort_values(by=columna_fecha).reset_index(drop=True)
                   
                    # Calcular la diferencia entre niveles consecutivos
                    df['Diferencia'] = df[columna_valor].diff()

                    # Calcular el delta de tiempo entre fechas consecutivas
                    df['Delta_Tiempo'] = df[columna_fecha].diff()
                
                    # Convertir la frecuencia esperada a un timedelta
                    freq_timedelta = pd.to_timedelta(periodos)
                    
                    # Identificar valores no consecutivos
                    df['Es_Consecutivo'] = df['Delta_Tiempo'] <= freq_timedelta
                    
                                        
                    # Calcular evaporación solo para registros consecutivos con disminuciones
                    df['Evaporacion'] = np.where(
                        df['Es_Consecutivo'] & (df['Diferencia'] < 0),
                        df['Diferencia'].abs(),
                        np.nan
                    )

                    # Se filtran columnas relevantes para el resultado de mhmma según si es raw o qc
                    if 'Estado' in df.columns:
                        ev_i = df[['Station','Fecha','Evaporacion','Estado']]
                    else:
                        ev_i = df[['Station','Fecha','Evaporacion']]
                    sttn = df['Station'][0]
                    ev_i.to_csv(f'ev_i_{sttn}.csv')

                    return df[['Station','Fecha','Evaporacion']] # evaporacion_total,

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    ev_i = EV_I(dfC_c)
                else:
                    ev_i = EV_I(chunk_c)

                # Evaporación total horaria
                ev_i_c = ev_i.copy()
                def EV_TT_H(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                    
                    if periodos == 'H':
                        return df_c[['Station','Fecha','Evaporacion']]
                
                    if df_c is None or complet_hora is None:
                        return None
                
                    # Se filtran los datos que tienen la complititud mínima
                    df_filtrado = df_c.groupby([df_c.index.hour]).filter(complet_hora) #dfC.groupby([dfC.index.date]).filter(complet_dia)
                    ev_tt_h = df_filtrado[[columna_valor]].resample('h').sum()
                    ev_tt_h[columna_valor] = ev_tt_h[columna_valor].where(
                        df_filtrado[columna_valor].resample('h').count() >= cant_esperd_h * porc_min,
                        other=float('nan'))

                    return ev_tt_h
                
                # Ejemplo de uso de la función
                ev_tt_h = EV_TT_H(ev_i_c)
                
                # Evaporación total diaria
                ev_tt_h_c = ev_tt_h.copy()
                def EV_TT_D(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Revisar resultado anterior
                    if df.empty:
                        return df
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                    cant_esperd_d = 24
                    def complet_dia(sub_df):
                        return len(sub_df) >= cant_esperd_d * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
                    # Calcular la suma diaria de evaporación
                    ev_tt_d = df_filtrado[[columna_valor]].resample('D').sum().round(4)
                    # Aplicar filtro de completitud diaria
                    ev_tt_d[columna_valor] = ev_tt_d[columna_valor].where(
                        df_filtrado[columna_valor].resample('D').count() >= cant_esperd_d * porc_min,
                        other=float('nan'))
                    
                    return ev_tt_d
                
                ev_tt_d = EV_TT_D(ev_tt_h_c)

                # Evaporación total mensual
                ev_tt_d_c = ev_tt_d.copy()
                def EV_TT_M(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Revisar resultado anterior
                    if df.empty:
                        return df
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                
                    # Cantidad días mes
                    days_in_month = df.index.to_series().dt.days_in_month
                    days_in_month = days_in_month.resample('ME').first()
                    
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_mes(sub_df):
                        mes = sub_df.index[0].month
                        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                        return len(sub_df) >= total_esperado * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                    # Calcular la suma diaria de evaporación
                    ev_tt_m = df_filtrado[[columna_valor]].resample('ME').sum().round(4)
                
                    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
                    counts = df.groupby([df.index.year, df.index.month])[columna_valor].count()
                    counts.index = pd.to_datetime(['{}-{}'.format(i[0], i[1]) for i in counts.index], format='%Y-%m')
                    counts = counts.resample('ME').sum()
                    ev_tt_m[columna_valor] = ev_tt_m[columna_valor].where(counts >= days_in_month * porc_min, other=float('nan'))
                    
                    return ev_tt_m
                
                ev_tt_m = EV_TT_M(ev_tt_d_c)
                
                # Evaporación total anual
                ev_tt_m_c = ev_tt_m.copy()
                def EV_TT_A(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Revisar resultado anterior
                    if df.empty:
                        return df
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                   
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                    
                    cant_esperd_a = 12
                    def complet_anio(sub_df):
                        return len(sub_df) >= cant_esperd_a * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                    # Calcular la suma diaria de evaporación
                    ev_tt_a = df_filtrado[[columna_valor]].resample('YE').sum().round(4)
                
                    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
                    counts = df.groupby(df.index.year)[columna_valor].count()
                    counts.index = pd.to_datetime(['{}-01'.format(i) for i in counts.index], format='%Y-%m')
                    counts = counts.resample('YE').sum()
                    ev_tt_a[columna_valor] = ev_tt_a[columna_valor].where(counts >= 12 * porc_min, other=float('nan'))
                
                    return ev_tt_a
                
                ev_tt_a = EV_TT_A(ev_tt_m_c)
                
                ## Derivadas medias horarias diarias, mensuales y anuales
                # Evaporación media horaria
                def EV_MEDIA_H(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                    
                    if periodos == 'H':
                        return df_c[['Station','Fecha','Evaporacion']]
                
                    if df_c is None or complet_hora is None:
                        return None
                
                    # Se filtran los datos que tienen la complititud mínima
                    df_filtrado = df_c.groupby([df_c.index.hour]).filter(complet_hora) #dfC.groupby([dfC.index.date]).filter(complet_dia)
                    ev_media_h = df_filtrado[[columna_valor]].resample('h').mean().round(4)
                
                    return ev_media_h

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    ev_media_h = EV_MEDIA_H(ev_i)
                else:
                    ev_media_h = EV_MEDIA_H(ev_i)
                
                # Evaporación media diaria
                ev_media_h_c = ev_media_h.copy()
                def EV_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Revisar resultado anterior
                    if df.empty:
                        return df
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                    cant_esperd_d = 24
                    def complet_dia(sub_df):
                        return len(sub_df) >= cant_esperd_d * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
                    # Calcular la suma diaria de evaporación
                    ev_media_d = df_filtrado[[columna_valor]].resample('D').mean().round(4)
                    # Aplicar filtro de completitud diaria
                    ev_media_d[columna_valor] = ev_media_d[columna_valor].where(
                        df_filtrado[columna_valor].resample('D').count() >= cant_esperd_d * porc_min,
                        other=float('nan'))
                
                    return ev_media_d
                
                ev_media_d = EV_MEDIA_D(ev_media_h_c)
                
                # Evaporación media mensual
                ev_media_d_c = ev_media_d.copy()
                def EV_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Revisar resultado anterior
                    if df.empty:
                        return df
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                
                    # Cantidad días mes
                    days_in_month = df.index.to_series().dt.days_in_month
                    days_in_month = days_in_month.resample('ME').first()
                    
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_mes(sub_df):
                        mes = sub_df.index[0].month
                        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                        return len(sub_df) >= total_esperado * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                    # Calcular la suma diaria de evaporación
                    ev_media_m = df_filtrado[[columna_valor]].resample('ME').mean().round(4)
                
                    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
                    counts = df.groupby([df.index.year, df.index.month])[columna_valor].count()
                    counts.index = pd.to_datetime(['{}-{}'.format(i[0], i[1]) for i in counts.index], format='%Y-%m')
                    counts = counts.resample('ME').sum()
                    ev_media_m[columna_valor] = ev_media_m[columna_valor].where(counts >= days_in_month * porc_min, other=float('nan'))
                    
                    return ev_media_m
                
                ev_media_m = EV_MEDIA_M(ev_media_d_c)
                
                # Evaporación media anual
                ev_media_m_c = ev_media_m.copy()
                def EV_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Evaporacion', porc_min=0.7):
                    df.reset_index(inplace=True)
                    # Revisar resultado anterior
                    if df.empty:
                        return df
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                   
                    # Establecer la columna de fecha como índice
                    df.set_index(columna_fecha, inplace=True)
                    
                    cant_esperd_a = 12
                    def complet_anio(sub_df):
                        return len(sub_df) >= cant_esperd_a * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                    # Calcular la suma diaria de evaporación
                    ev_media_a = df_filtrado[[columna_valor]].resample('YE').mean().round(4)
                
                    # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
                    counts = df.groupby(df.index.year)[columna_valor].count()
                    counts.index = pd.to_datetime(['{}-01'.format(i) for i in counts.index], format='%Y-%m')
                    counts = counts.resample('YE').sum()
                    ev_media_a[columna_valor] = ev_media_a[columna_valor].where(counts >= 12 * porc_min, other=float('nan'))
                
                    return ev_media_a
                
                ev_media_a = EV_MEDIA_A(ev_media_m_c)
                
                # ###-- Derivados máximos y mínimos
                # #Humedad relativa del aire a 10 cm  mínima diaria
                # def HRS30_MN_H(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_hora is None:
                #         return None
                
                #     # Filtrar los días que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     horas_validas = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_hora).index
                    
                #     # Filtrar el DataFrame original para incluir solo los días válidos
                #     horas_validas = pd.to_datetime(horas_validas).normalize()
                #     df_filtrado = df_c[df_c.index.normalize().isin(horas_validas)]
                    
                #     # Encontrar el valor mínimo por cada día válido
                #     idx_minimos_hora = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmin()
                #     mn_d = df_filtrado.loc[idx_minimos_hora]
                    
                #     # Eliminar la columna temporal antes de retornar el resultado
                #     if 'Fecha_temp' in mn_d.columns:
                #         mn_d.drop(columns=['Fecha_temp'], inplace=True)
                    
                #     return mn_d[[columna_valor]]

                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mn_h = HRS30_MN_H(dfC_c)
                # else:
                #     df_mn_h = HRS30_MN_H(chunk_c)
                    
                # #Humedad relativa del aire a 10 cm  mínima diaria
                # def HRS30_MX_H(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_hora is None:
                #         return None
                
                #     # Filtrar los días que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     horas_validas = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_hora).index
                    
                #     # Filtrar el DataFrame original para incluir solo los días válidos
                #     horas_validas = pd.to_datetime(horas_validas).normalize()
                #     df_filtrado = df_c[df_c.index.normalize().isin(horas_validas)]
                    
                #     # Encontrar el valor mínimo por cada día válido
                #     idx_maximos_hora = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmax()
                #     mx_h = df_filtrado.loc[idx_maximos_hora]
                    
                #     # Eliminar la columna temporal antes de retornar el resultado
                #     if 'Fecha_temp' in mx_h.columns:
                #         mx_h.drop(columns=['Fecha_temp'], inplace=True)
                    
                #     return mx_h[[columna_valor]]

                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mx_h = HRS30_MX_H(dfC_c)
                # else:
                #     df_mx_h = HRS30_MX_H(chunk_c)
                
                # def HRS30_MN_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo,_,complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_dia is None:
                #         return None
                
                #     # Filtrar los días que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
                    
                #     # Filtrar el DataFrame original para incluir solo los días válidos
                #     dias_validos = pd.to_datetime(dias_validos).normalize()
                #     df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
                    
                #     # Encontrar el valor mínimo por cada día válido
                #     idx_minimos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmin()
                #     mn_d = df_filtrado.loc[idx_minimos_dia]
                    
                #     # Eliminar la columna temporal antes de retornar el resultado
                #     if 'Fecha_temp' in mn_d.columns:
                #         mn_d.drop(columns=['Fecha_temp'], inplace=True)
                    
                #     return mn_d[[columna_valor]]

                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mn_d = HRS30_MN_D(dfC_c)
                # else:
                #     df_mn_d = HRS30_MN_D(chunk_c)
                
                # # Humedad relativa del aire a 10 cm  máxima diaria
                # def HRS30_MX_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo,_,complet_dia,_,_ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_dia is None:
                #         return None
                
                #     # Filtrar los días que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
                #     # Filtrar el DataFrame original para incluir solo los días válidos
                #     dias_validos = pd.to_datetime(dias_validos).normalize()
                #     df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
                #     # Encontrar el valor mínimo por cada día válido
                #     idx_maximos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmax()
                #     mx_d = df_filtrado.loc[idx_maximos_dia]
                    
                #     # Eliminar la columna temporal antes de retornar el resultado
                #     if 'Fecha_temp' in mx_d.columns:
                #         mx_d.drop(columns=['Fecha_temp'], inplace=True)
                    
                #     return mx_d[[columna_valor]]
                
                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mx_d = HRS30_MX_D(dfC_c)
                # else:
                #     df_mx_d = HRS30_MX_D(chunk_c)
                
                # # Humedad relativa del aire a 10 cm  mínima mensual
                # def HRS30_MN_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo,_,_,complet_mes,_ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_mes is None:
                #         return None
                
                #     # Filtrar los meses que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     # Eliminar duplicados en el índice
                #     df_c = df_c[~df_c.index.duplicated(keep='first')]
                #     meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')
                
                #     # Filtrar el DataFrame original para incluir solo los meses válidos
                #     df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]
                
                #     # Encontrar el valor mínimo por cada mes válido
                #     idx_minimos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmin()
                #     min_m = df_filtrado.loc[idx_minimos_mes]
                
                #     # Eliminar las columnas temporales antes de retornar el resultado
                #     if 'Fecha_temp' in min_m.columns:
                #         min_m.drop(columns=['Fecha_temp'], inplace=True)
                
                #     return min_m[[columna_valor]]

                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mn_m = HRS30_MN_M(dfC_c)
                # else:
                #     df_mn_m = HRS30_MN_M(chunk_c)
                
                # # Humedad relativa del aire a 10 cm  máxima mensual
                # def HRS30_MX_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo,_,_,complet_mes,_ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_mes is None:
                #         return None
                
                #     # Filtrar los meses que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     # Eliminar duplicados en el índice
                #     df_c = df_c[~df_c.index.duplicated(keep='first')]
                #     meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')
                
                #     # Filtrar el DataFrame original para incluir solo los meses válidos
                #     df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]
                
                #     # Encontrar el valor mínimo por cada mes válido
                #     idx_maximos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmax()
                #     max_m = df_filtrado.loc[idx_maximos_mes]
                
                #     # Eliminar las columnas temporales antes de retornar el resultado
                #     if 'Fecha_temp' in max_m.columns:
                #         max_m.drop(columns=['Fecha_temp'], inplace=True)
                
                #     return max_m[[columna_valor]]

                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mx_m = HRS30_MX_M(dfC_c)
                # else:
                #     df_mx_m = HRS30_MX_M(chunk_c)
                
                # # Humedad relativa del aire a 10 cm mínima anual
                # def HRS30_MN_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo,_,_,_,complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_anio is None:
                #         return None
                
                #     # Filtrar los meses que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     # Eliminar duplicados en el índice
                #     df_c = df_c[~df_c.index.duplicated(keep='first')]
                #     anios_validos = df_c.groupby(df_c.index.to_period('Y')).filter(complet_anio).index.to_period('Y')
                
                #     # Filtrar el DataFrame original para incluir solo los meses válidos
                #     df_filtrado = df_c[df_c.index.to_period('Y').isin(anios_validos)]
                
                #     # Encontrar el valor mínimo por cada mes válido
                #     idx_minimos_anio = df_filtrado.groupby(df_filtrado.index.to_period('Y'))[columna_valor].idxmin()
                #     min_a = df_filtrado.loc[idx_minimos_anio]
                
                #     # Eliminar las columnas temporales antes de retornar el resultado
                #     if 'Fecha_temp' in min_a.columns:
                #         min_a.drop(columns=['Fecha_temp'], inplace=True)
                
                #     return min_a[[columna_valor]]

                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mn_a = HRS30_MN_A(dfC_c)
                # else:
                #     df_mn_a = HRS30_MN_A(chunk_c)
                
                # # Humedad relativa del aire a 10 cm  máxima anual
                # def HRS30_MX_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.7):
                #     # Convertir la columna de fecha a datetime si aún no lo es
                #     if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                #         df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                #     # Se llama la función de procesamiento de frecuencias
                #     df_c, periodo,_,_,_,complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                #     if df_c is None or complet_anio is None:
                #         return None
                
                #     # Filtrar los meses que tienen suficientes datos
                #     df_c['Fecha_temp'] = df_c.index
                #     # Eliminar duplicados en el índice
                #     df_c = df_c[~df_c.index.duplicated(keep='first')]
                #     anios_validos = df_c.groupby(df_c.index.to_period('Y')).filter(complet_anio).index.to_period('Y')
                
                #     # Filtrar el DataFrame original para incluir solo los meses válidos
                #     df_filtrado = df_c[df_c.index.to_period('Y').isin(anios_validos)]
                
                #     # Encontrar el valor mínimo por cada mes válido
                #     idx_maximos_anio = df_filtrado.groupby(df_filtrado.index.to_period('Y'))[columna_valor].idxmax()
                #     max_a = df_filtrado.loc[idx_maximos_anio]
                
                #     # Eliminar las columnas temporales antes de retornar el resultado
                #     if 'Fecha_temp' in max_a.columns:
                #         max_a.drop(columns=['Fecha_temp'], inplace=True)
                
                #     return max_a[[columna_valor]]
                    
                # # Se llama el archivo original según si es dato crudo o con qc
                # if 'Estado' in chunk.columns:
                #     df_mx_a = HRS30_MX_A(dfC_c)
                # else:
                #     df_mx_a = HRS30_MX_A(chunk_c)
        
                ### Se crea un nuevo archivo Excel con openpyxl
                wb = Workbook()
                sheets = {
                    #'EV_I': ev_i,
                    'EV_TT_H': ev_tt_h, 'EV_TT_D': ev_tt_d, 
                    'EV_TT_M': ev_tt_m, 'EV_TT_A': ev_tt_a, 
                    'EV_MEDIA_H': ev_media_h, 'EV_MEDIA_D': ev_media_d, 
                    'EV_MEDIA_M': ev_media_m, 'EV_MEDIA_A': ev_media_a,
                    # 'HRS30_MEDIA_H': dfhrs30_med_h, 'HRS30_MEDIA_D': dfhrs30_med_d,
                    # 'HRS30_MEDIA_M': dfhrs30_med_m, 'HRS30_MEDIA_A': dfhrs30_med_a, 
                    # 'HRS30_MN_H': df_mn_h,'HRS30_MX_H': df_mx_h,
                    # 'HRS30_MN_D': df_mn_d,'HRS30_MX_D': df_mx_d, 
                    # 'HRS30_MN_M': df_mn_m,'HRS30_MX_M': df_mx_m, 
                    # 'HRS30_MN_A': df_mn_a,'HRS30_MX_A': df_mx_a 
                }
                
                # Si el workbook todavía tiene la hoja por defecto, se elimina
                if "Sheet" in wb.sheetnames:
                    del wb["Sheet"]
                
                for sheet_name, data in sheets.items():
                    ws = wb.create_sheet(title=sheet_name)
                    
                    # Agregamos los datos al Excel
                    if data is None:
                        pass
                    else:
                        for r_idx, row in enumerate(dataframe_to_rows(data, index=True, header=True), 1):
                            for c_idx, value in enumerate(row, 1):
                                ws.cell(row=r_idx, column=c_idx, value=value)
                    
                    # Crear una gráfica
                    chart = LineChart()
                    chart.title = sheet_name
                    chart.style = 5
                    chart.y_axis.title = 'Evaporación (mm)'
                    chart.x_axis.title = 'Fecha'
                    
                    # Establecer datos para la gráfica
                    max_row = ws.max_row
                    values = Reference(ws, min_col=2, min_row=2, max_col=2, max_row=max_row)
                    dates = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=max_row)
                    chart.add_data(values, titles_from_data=True)
                    chart.set_categories(dates)
                    
                    # Quitar la leyenda
                    chart.legend = None+
                    
                    # Cambiar el grosor de la línea a 0.5 puntos (equivalente a 50 centésimas de punto)
                    for series in chart.series:
                        series.graphicalProperties.line.width = 50
                        series.graphicalProperties.line.solidFill = "3498db" 
                        
                    # Posicionar la gráfica en el Excel
                    ws.add_chart(chart, "E3")
                
                # Nombres archivos
                if 'Estado' in chunk.columns:
                    nombre_archivo_salida = os.path.join(carpeta, archivo[:22] + '_deriv.xlsx')
                else:
                    nombre_archivo_salida = os.path.join(carpeta, archivo[:19] + '_deriv.xlsx')
                #nombre_archivo_salida = os.path.join(carpeta, archivo[:22] + '_deriv.xlsx') #archivo[:19] el original #archivo[:22] con qc
                
                # Guardar el archivo Excel
                wb.save(nombre_archivo_salida)

In [9]:
calc_deriv_EV('../../OE_3_QC_Variables/5_Evaporacion/QCResult_EV/')

C:\Users\palvarez\AppData\Local\Temp\ipykernel_16420\4222087669.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


Error en el archivo Estacion_0023195040_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0024025050_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0026255030_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0027015320_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0035215020_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0046015030_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0052055160_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0054017040_qc.csv: dfC está vacío. Saltando al siguiente archivo.


In [12]:
calc_deriv_HRS30('../../../OE_3_QC_Variables/4_HumedadSuelo/HRS30/RawUnmodified_HRS30/')

----

## Función cálculo masivo de derivadas y alistamiento Cassandra

In [73]:
def calc_deriv_EV_QC(carpeta):#, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se crean carpetas para cada tipo de DataFrame si no existen
    carpetas_salidas = ['EV_TT_H_QC', 'EV_TT_D_QC', 'EV_TT_M_QC', 'EV_TT_A_QC']#,
                        #'EV_MN_H_QC', 'EV_MX_H_QC', 'EV_MN_D_QC', 'EV_MX_D_QC',
                        #'EV_MN_M_QC', 'EV_MX_M_QC', 'EV_MN_A_QC', 'EV_MX_A_QC']

    for cs in carpetas_salidas:
        os.makedirs(os.path.join(carpeta, cs), exist_ok=True)
    
    # Procesar cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
            df = pd.read_csv(ruta_archivo, encoding='latin-1')

            # Procesar la fecha y filtrar según estado, como en el ejemplo original
            try:
                df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S.%f')
            except ValueError:
                df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S')
                  
            try:
                dfC = df[df['state'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))] # Se comenta si es de crudos
                station_value = dfC['station'].values[0] #df['station'].values[0]
            except IndexError:
                print(f"Error en el archivo {archivo}: dfC está vacío. Saltando al siguiente archivo.")
                continue  # Sale del bucle de chunks y continúa con el siguiente archivo

            dfC_c = dfC.copy()
            # Cálculo de la evaporación instantánea
            def EV_I(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.7):
                """
                Calcula la evaporación en un tanque a partir de datos de nivel, considerando solo las disminuciones de nivel.
                """
                # Asegurarnos de que la columna de fecha sea datetime
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])

                # Se llama la función de procesamiento de frecuencias
                df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

                # Asegurarse de que 'periodos' tenga un número antes de la unidad
                if periodos.isalpha():
                    periodos = '1' + periodos
                    
                # Ordenar por fecha por seguridad
                df = df.sort_values(by=columna_fecha).reset_index(drop=True)
               
                # Calcular la diferencia entre niveles consecutivos
                df['Diferencia'] = df[columna_valor].diff()

                # Calcular el delta de tiempo entre fechas consecutivas
                df['Delta_Tiempo'] = df[columna_fecha].diff()
            
                # Convertir la frecuencia esperada a un timedelta
                freq_timedelta = pd.to_timedelta(periodos)
                
                # Identificar valores no consecutivos
                df['Es_Consecutivo'] = df['Delta_Tiempo'] <= freq_timedelta
                                   
                # Calcular evaporación solo para registros consecutivos con disminuciones
                df[columna_valor] = np.where(
                    df['Es_Consecutivo'] & (df['Diferencia'] < 0),
                    df['Diferencia'].abs(),
                    np.nan
                )

                # Se filtran columnas relevantes para el resultado de mhmma según si es raw o qc
                ev_i = df[['station','event_time',columna_valor]]
                
                
                return ev_i # evaporacion_total,

            # Se llama el archivo original según si es dato crudo o con qc
            ev_i = EV_I(dfC_c)
            
            # Evaporación total horaria
            def EV_TT_H_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.7):  
                # Se llama la función de procesamiento de frecuencias
                df_c, periodos, cant_esperd_h, _, _, _, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

                if periodos == 'h':
                    df_c.reset_index(inplace=True)
                    df_c['label'] = 'EV_TT_H_QC'
                    return df_c[['station','label','event_time','event_value']]
            
                if df_c is None or complet_hora is None:
                    return None
            
                # Se filtran los datos que tienen la complititud mínima
                df_filtrado = df_c.groupby([df_c.index.hour]).filter(complet_hora) #dfC.groupby([dfC.index.date]).filter(complet_dia)
                EV_60_tt_h = df_filtrado[[columna_valor]].resample('h').sum()
                EV_60_tt_h[columna_valor] = EV_60_tt_h[columna_valor].where(
                    df_filtrado[columna_valor].resample('h').count() >= cant_esperd_h * porc_min,
                    other=float('nan'))

                # Cambio de contenido de columnas
                EV_60_tt_h['station'] = df_c['station'].iloc[0]
                EV_60_tt_h['station'] = EV_60_tt_h['station'].astype('int64')
                EV_60_tt_h['label'] = 'EV_TT_H_QC'
                
                # Reset index
                EV_60_tt_h.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                EV_60_tt_h = EV_60_tt_h[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                EV_60_tt_h['event_time'] = pd.to_datetime(EV_60_tt_h['event_time'])
            
                return EV_60_tt_h

            dfev_tt_h = EV_TT_H_QC(ev_i)
            
            # Evaporación total diaria
            def EV_TT_D_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.7):
                if df[columna_valor].empty:
                    return None
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                # Ajustar la hora de cada registro para que corresponda al rango deseado
                df[columna_fecha] = df[columna_fecha] - pd.Timedelta(hours=1)
                # Establecer la columna de fecha como índice
                df.set_index(columna_fecha, inplace=True)
                
                # Calcular el total de registros esperados por día pluviométrico
                total_esperado_por_dia = 24
                # Función para verificar si un día pluviométrico tiene suficientes datos
                def complet_dia(sub_df):
                    return len(sub_df) >= total_esperado_por_dia * porc_min
            
                # Filtrar los días con suficientes datos y calcular el promedio diario
                df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por días válidos. Regresando {archivo[:19]} vacío.")
                    return None
                    
                # Se calcula la media
                EV_60_tt_d = df_filtrado[[columna_valor]].resample('D').sum()

                # Llenado de vacíos cuando no superan pruebas
                EV_60_tt_d[columna_valor] = EV_60_tt_d[columna_valor].where(
                    df_filtrado[columna_valor].resample('D').count() >= total_esperado_por_dia * porc_min,
                    other=float('nan'))

                if EV_60_tt_d.empty:
                    print(f"DataFrame vacío después de filtrar por días válidos. Regresando {archivo[:19]} vacío.")
                    return None

                # Cambio de contenido de columnas
                EV_60_tt_d['station'] = df['station'].iloc[0]
                EV_60_tt_d['station'] = EV_60_tt_d['station'].astype('int64')
                EV_60_tt_d['label'] = 'EV_TT_D_QC'
                
                # Reset index
                EV_60_tt_d.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                EV_60_tt_d = EV_60_tt_d[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                EV_60_tt_d['event_time'] = pd.to_datetime(EV_60_tt_d['event_time'])
            
                return EV_60_tt_d

            dfev_tt_d = EV_TT_D_QC(dfev_tt_h)
            
            # Evaporación total mensual
            dfev_tt_d_c = dfev_tt_d.copy()
            def EV_TT_M_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.7):    
                if df[columna_valor].empty:
                    return None
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                # Establecer la columna 'event_time' como índice
                df.set_index(columna_fecha, inplace=True)
                
                days_in_month = df.index.to_series().dt.days_in_month
                days_in_month = days_in_month.resample('ME').first()
                
                # Función para verificar si una hora específica tiene suficientes datos
                def complet_mes(sub_df):
                    mes = sub_df.index[0].month
                    total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                    return len(sub_df) >= total_esperado * porc_min
            
                # Luego de establecer el índice, aplicar resample
                df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por meses válidos. Regresando {archivo[:19]} vacío.")
                    return None
                    
                # Se calcula la media 
                EV_60_tt_m = df_filtrado[['event_value']].resample('ME').sum()

                # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
                counts = df.groupby([df.index.year, df.index.month])[columna_valor].count()
                counts.index = pd.to_datetime(['{}-{}'.format(i[0], i[1]) for i in counts.index], format='%Y-%m')
                counts = counts.resample('ME').sum()
                EV_60_tt_m[columna_valor] = EV_60_tt_m[columna_valor].where(counts >= days_in_month * porc_min, other=float('nan'))
                if EV_60_tt_m.empty:
                    print(f"DataFrame vacío después de filtrar por meses válidos. Regresando {archivo[:19]} vacío.")
                    return None
                
                # Cambio de contenido de columnas
                EV_60_tt_m['station'] = df['station'].iloc[0]
                EV_60_tt_m['station'] = EV_60_tt_m['station'].astype('int64')
                EV_60_tt_m['label'] = 'EV_TT_M_QC'
                
                # Reset index
                EV_60_tt_m.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                EV_60_tt_m = EV_60_tt_m[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                EV_60_tt_m['event_time'] = pd.to_datetime(EV_60_tt_m['event_time'])
                
                return EV_60_tt_m
            
            dfev_tt_m = EV_TT_M_QC(dfev_tt_d_c)
            
            # Evaporación total anual
            dfev_tt_m_c = dfev_tt_m.copy()
            def EV_TT_A_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.7):
                if df[columna_valor].empty:
                    return None
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                # Función para verificar si una hora específica tiene suficientes datos
                def complet_anio(sub_df):
                    return len(sub_df) >= 12 * porc_min
                   
                # Antes de resample, establecer la columna 'event_time' como índice
                df.set_index(columna_fecha, inplace=True)
            
                # Luego de establecer el índice, aplicar resample
                df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por años válidos. Regresando {archivo[:19]} vacío.")
                    return None
                    
                # Se calcula la media    
                EV_60_tt_a = df_filtrado[['event_value']].resample('YE').sum()

                # Reemplazar sumas de 0 con NaN si no cumplen con el porcentaje mínimo
                counts = df.groupby(df.index.year)[columna_valor].count()
                counts.index = pd.to_datetime(['{}-01'.format(i) for i in counts.index], format='%Y-%m')
                counts = counts.resample('YE').sum()
                EV_60_tt_a[columna_valor] = EV_60_tt_a[columna_valor].where(counts >= 12 * porc_min, other=float('nan'))
                if EV_60_tt_a.empty:
                    print(f"DataFrame vacío después de filtrar por años válidos. Regresando {archivo[:19]} vacío.")
                    return None

                # Cambio de contenido de columnas
                EV_60_tt_a['station'] = df['station'].iloc[0]
                EV_60_tt_a['station'] = EV_60_tt_a['station'].astype('int64')
                EV_60_tt_a['label'] = 'EV_TT_A_QC'
                
                # Reset index
                EV_60_tt_a.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                EV_60_tt_a = EV_60_tt_a[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                EV_60_tt_a['event_time'] = pd.to_datetime(EV_60_tt_a['event_time'])
                
                return EV_60_tt_a
            
            dfev_tt_a = EV_TT_A_QC(dfev_tt_m_c)

            if not dfev_tt_h.empty:
                dfev_tt_h.to_csv(os.path.join(carpeta, 'EV_TT_H_QC', f'{archivo[:19]}.csv'), index=False)
            if not dfev_tt_d.empty:
                dfev_tt_d.to_csv(os.path.join(carpeta, 'EV_TT_D_QC', f'{archivo[:19]}.csv'), index=False)
            if not dfev_tt_m.empty:
                dfev_tt_m.to_csv(os.path.join(carpeta, 'EV_TT_M_QC', f'{archivo[:19]}.csv'), date_format='%Y-%m', index=False)
            if not dfev_tt_a.empty:
                dfev_tt_a.to_csv(os.path.join(carpeta, 'EV_TT_A_QC', f'{archivo[:19]}.csv'), date_format='%Y',index=False)

In [75]:
calc_deriv_EV_QC('../../OE_3_QC_Variables/5_Evaporacion/ReadyToCassandraFiles_EV/')

Error en el archivo Estacion_0023195040_qc.csv: dfC está vacío. Saltando al siguiente archivo.
DataFrame vacío después de filtrar por días válidos. Regresando Estacion_0024015513 vacío.


AttributeError: 'NoneType' object has no attribute 'copy'

### Cálculo promedios horarios mensuales multianuales

In [13]:
def calc_hmMa_EV(carpeta, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
            
            for chunk in reader:
                # Se generan dataframes analizados
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                try:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                    #chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')
                except ValueError:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')
                    chunk = chunk[~chunk['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO0','0PAT','0PER']]))]

                # Se hace la agrupación para cálculo de medias horarias mensuales multianuales
                #hym_ma = chunk['Valor'].groupby(by =[chunk["Fecha"].dt.month, chunk["Fecha"].dt.hour]).mean().unstack(level=0)
                hym_ma = chunk['Evaporacion'].groupby(by =[chunk["Fecha"].dt.month, chunk["Fecha"].dt.hour]).mean().unstack(level=0)

                # Crear un archivo Excel y agregar los datos
                wb = Workbook()
                ws = wb.active
                ws.title = "Datos"
                
                # Agregar datos al archivo Excel
                for r in dataframe_to_rows(hym_ma.reset_index(), index=False, header=True):
                    ws.append(r)
                
                # Crear la gráfica de dispersión
                chart = ScatterChart()
                chart.title = "Valores Promedio por Hora y Mes"
                chart.style = 13
                chart.x_axis.title = 'Hora del día'
                chart.y_axis.title = 'Valor'
                
                # Aumentar el tamaño del gráfico
                chart.width = 20  # Anchura del gráfico (pulgadas)
                chart.height = 12  # Altura del gráfico (pulgadas)
                
                # Fijar el máximo valor del eje x
                chart.x_axis.scaling.max = 23
                chart.x_axis.scaling.min = 0
                chart.x_axis.majorUnit = 1
                
                # Agregar series a la gráfica
                colors = ['1F77B4', 'FF7F0E', '2CA02C', 'D62728', '9467BD', '8C564B', 'E377C2', '7F7F7F', 'BCBD22', '17BECF', 'AEC7E8', 'FFBB78']
                for i in range(2, 14):  # Columnas B a M (meses 1 a 12)
                    xvalues = Reference(ws, min_col=1, min_row=2, max_row=25)
                    yvalues = Reference(ws, min_col=i, min_row=1, max_row=25)
                    series = Series(yvalues, xvalues, title_from_data=True)
                    series.graphicalProperties.line.solidFill = colors[i % len(colors)]  # Asignar colores a las líneas
                    series.graphicalProperties.line.width = 30000  # Ajustar el grosor de las líneas
                    series.marker.symbol = 'circle'  # Cambiar el marcador a círculo
                    series.marker.size = 5
                    series.marker.graphicalProperties.solidFill = colors[i % len(colors)]  # Cambiar el color del marcador
                    chart.series.append(series)
                
                # Insertar la gráfica en la hoja de cálculo
                ws.add_chart(chart, "O2")

                # Nombres archivos
                nombre_archivo_salida = os.path.join(carpeta, archivo[:13] + '_hm_ma.xlsx') #archivo[:22] el de qc
                # Guardar el archivo Excel
                wb.save(nombre_archivo_salida)

In [19]:
calc_hmMa_EV('EV_i_qc')
#calc_hmMa_EV('EV_i')

In [ ]:
calc_hmMa_EV('../../OE_3_QC_Variables/5_Evaporacion/RawUnmodified_EV/')